In [2]:
import pandas as pd

In [8]:
df = pd.read_csv('235217340862dat.txt',skiprows = 2)

In [9]:
df.head()

,725053,94728,20140101,0051,7,FM-15,,290,5,N,...,9.519,999.57,99999.7,99.1,99.2,+9999,9,9.1,MET10412/31/13 19:51:03 METAR KNYC 010051Z 29009G20KT 10SM CLR M01/M11 A3021 RMK AO2 SLP221 T10111111 RTX FIBI,
0,725053,94728,20140101,151,7,FM-15,,290,5,V,...,9,999,99999,99,99,9999,9,9,MET10112/31/13 20:51:03 METAR KNYC 010151Z 290...,...
1,725053,94728,20140101,251,7,FM-15,,300,5,N,...,9,999,99999,99,99,9999,9,9,MET11012/31/13 21:51:03 METAR KNYC 010251Z 300...,...
2,725053,94728,20140101,351,7,FM-15,,999,9,V,...,9,999,99999,99,99,9999,9,9,MET10412/31/13 22:51:03 METAR KNYC 010351Z VRB...,...
3,725053,94728,20140101,451,7,FM-15,,290,5,N,...,9,999,99999,99,99,9999,9,9,MET11412/31/13 23:51:03 METAR KNYC 010451Z 290...,...
4,725053,94728,20140101,459,6,SOM,,999,9,9,...,9,999,99999,99,99,9999,9,9,SOM796PCP MTOT:4.85 PCP GT 24HRS:1.45 DATE(S):...,...
